# PySpark SparkSQL Analytics Functions


In [1]:
%%html
<style>
table {float:left}
</style>

In [2]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
</style>

In [3]:
import os
import sys
import gc
from datetime import (
    datetime,
    date
)

#  Environemnt Variables

## Hadoop

In [4]:
os.environ['HADOOP_CONF_DIR'] = "/opt/hadoop/hadoop-3.2.2/etc/hadoop"

In [5]:
%%bash
export HADOOP_CONF_DIR="/opt/hadoop/hadoop-3.2.2/etc/hadoop"
ls $HADOOP_CONF_DIR | head -n 5

capacity-scheduler.xml
configuration.xsl
container-executor.cfg
core-site.xml
core-site.xml.48132.2022-02-15@12:29:41~


## PYTHONPATH

Refer to the **pyspark** modules to load from the ```$SPARK_HOME/python/lib``` in the Spark installation.

* [PySpark Getting Started](https://spark.apache.org/docs/latest/api/python/getting_started/install.html)

> Ensure the SPARK_HOME environment variable points to the directory where the tar file has been extracted. Update PYTHONPATH environment variable such that it can find the PySpark and Py4J under SPARK_HOME/python/lib. One example of doing this is shown below:

```
export PYTHONPATH=$(ZIPS=("$SPARK_HOME"/python/lib/*.zip); IFS=:; echo "${ZIPS[*]}"):$PYTHONPATH
```

Alternatively install **pyspark** with pip or conda locally which installs the Spark runtime libararies (for standalone).

* [Can PySpark work without Spark?](https://stackoverflow.com/questions/51728177/can-pyspark-work-without-spark)

> As of v2.2, executing pip install pyspark will install Spark. If you're going to use Pyspark it's clearly the simplest way to get started. On my system Spark is installed inside my virtual environment (miniconda) at lib/python3.6/site-packages/pyspark/jars  
> PySpark has a Spark installation installed. If installed through pip3, you can find it with pip3 show pyspark. Ex. for me it is at ~/.local/lib/python3.8/site-packages/pyspark. This is a standalone configuration so it can't be used for managing clusters like a full Spark installation.

In [6]:
# os.environ['PYTHONPATH'] = "/opt/spark/spark-3.1.2/python/lib/py4j-0.10.9-src.zip:/opt/spark/spark-3.1.2/python/lib/pyspark.zip"
sys.path.extend([
    "/opt/spark/spark-3.1.2/python/lib/py4j-0.10.9-src.zip",
    "/opt/spark/spark-3.1.2/python/lib/pyspark.zip"
])

## PySpark packages

Execute after the PYTHONPATH setup

In [7]:
import pyspark.sql 
from pyspark.sql.types import *
from pyspark.sql.functions import (
    col,
    avg,
    stddev,
    isnan,
    to_date,
    to_timestamp,
    hour,
)

# Data

Student schema from [Oracle SQL by Example](https://learning.oreilly.com/library/view/oracle-sql-by/9780137047345/ch06.html) located in ```./data/student```. 

In [8]:
%%bash
cd ./data/student
unzip -o student.zip

Archive:  student.zip
  inflating: COURSE_DATA_TABLE.csv   
  inflating: COURSE_REVENUE_DATA_TABLE.csv  
  inflating: EMPLOYEE_DATA_TABLE.csv  
  inflating: ENROLLMENT_DATA_TABLE.csv  
  inflating: GRADE_DATA_TABLE.csv    
  inflating: GRADE_TYPE_DATA_TABLE.csv  
  inflating: INSTRUCTOR_DATA_TABLE.csv  
  inflating: SECTION_DATA_TABLE.csv  
  inflating: SECTION_HISTORY_DATA_TABLE.csv  
  inflating: STUDENT_DATA_TABLE.csv  
  inflating: ZIPCODE_DATA_TABLE.csv  


In [9]:
%%bash
cd data/student/
hdfs dfs -mkdir -p student
hdfs dfs -put -f *.csv student

rm -rf *.csv

---
# Spark Session


In [10]:
from pyspark.sql import SparkSession

In [11]:
spark = SparkSession.builder\
    .master('yarn') \
    .config('spark.submit.deployMode', 'client') \
    .config('spark.debug.maxToStringFields', 100) \
    .config('spark.executor.memory', '2g') \
    .getOrCreate()

2022-02-20 17:44:28,578 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-02-20 17:44:31,714 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [12]:
NUM_CORES = 4
NUM_PARTITIONS = 3

spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set('spark.sql.legacy.timeParserPolicy', 'LEGACY')

# Stduent schema CSV 

* [SparkSQL CSV Files](https://spark.apache.org/docs/latest/sql-data-sources-csv.html)

> Spark SQL provides spark.read().csv("file_name") to read a file or directory of files in CSV format into Spark DataFrame, and dataframe.write().csv("path") to write to a CSV file. Function option() can be used to customize the behavior of reading or writing.

[SparkSession.read()](https://spark.apache.org/docs/latest/api/java/org/apache/spark/sql/SparkSession.html#read--) returns [DataFrameReader](https://spark.apache.org/docs/latest/api/java/org/apache/spark/sql/DataFrameReader.html) instance which has [option](https://spark.apache.org/docs/latest/api/java/org/apache/spark/sql/DataFrameReader.html#option-java.lang.String-boolean-) method by which we can specify CSV options.

The options are listed in [Data Source Option](https://spark.apache.org/docs/latest/sql-data-sources-csv.html#data-source-option)

In [31]:
course = spark.read\
    .option("header", "true")\
    .option("nullValue", "")\
    .option("inferSchema", "true")\
    .csv("student/COURSE_DATA_TABLE.csv")\
    .withColumn("CREATED_DATE", to_date(col('CREATED_DATE'), "dd-MMM-yy"))\
    .withColumn("MODIFIED_DATE", to_date(col('MODIFIED_DATE'), "dd-MMM-yy"))

course.printSchema()
course.createOrReplaceTempView("course")
course.show(3)

root
 |-- COURSE_NO: integer (nullable = true)
 |-- DESCRIPTION: string (nullable = true)
 |-- COST: integer (nullable = true)
 |-- PREREQUISITE: integer (nullable = true)
 |-- CREATED_BY: string (nullable = true)
 |-- CREATED_DATE: date (nullable = true)
 |-- MODIFIED_BY: string (nullable = true)
 |-- MODIFIED_DATE: date (nullable = true)

+---------+--------------------+----+------------+----------+------------+-----------+-------------+
|COURSE_NO|         DESCRIPTION|COST|PREREQUISITE|CREATED_BY|CREATED_DATE|MODIFIED_BY|MODIFIED_DATE|
+---------+--------------------+----+------------+----------+------------+-----------+-------------+
|       10| Technology Concepts|1195|        null|  DSCHERER|  2007-03-29|   ARISCHER|   2007-04-05|
|       20|Intro to Informat...|1195|        null|  DSCHERER|  2007-03-29|   ARISCHER|   2007-04-05|
|       25|Intro to Programming|1195|         140|  DSCHERER|  2007-03-29|   ARISCHER|   2007-04-05|
+---------+--------------------+----+------------+-

In [36]:
section = spark.read\
    .option("header", "true")\
    .option("nullValue", "")\
    .option("inferSchema", "true")\
    .csv("student/SECTION_DATA_TABLE.csv")\
    .withColumn("START_DATE_TIME", to_date(col('START_DATE_TIME'), "dd-MMM-yy"))\
    .withColumn("CREATED_DATE", to_date(col('CREATED_DATE'), "dd-MMM-yy"))\
    .withColumn("MODIFIED_DATE", to_date(col('MODIFIED_DATE'), "dd-MMM-yy"))

section.printSchema()
section.createOrReplaceTempView("section")
section.show(3)

root
 |-- SECTION_ID: integer (nullable = true)
 |-- COURSE_NO: integer (nullable = true)
 |-- SECTION_NO: integer (nullable = true)
 |-- START_DATE_TIME: date (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- INSTRUCTOR_ID: integer (nullable = true)
 |-- CAPACITY: integer (nullable = true)
 |-- CREATED_BY: string (nullable = true)
 |-- CREATED_DATE: date (nullable = true)
 |-- MODIFIED_BY: string (nullable = true)
 |-- MODIFIED_DATE: date (nullable = true)

+----------+---------+----------+---------------+--------+-------------+--------+----------+------------+-----------+-------------+
|SECTION_ID|COURSE_NO|SECTION_NO|START_DATE_TIME|LOCATION|INSTRUCTOR_ID|CAPACITY|CREATED_BY|CREATED_DATE|MODIFIED_BY|MODIFIED_DATE|
+----------+---------+----------+---------------+--------+-------------+--------+----------+------------+-----------+-------------+
|        79|      350|         3|     2007-04-14|    L509|          107|      25|  CBRENNAN|  2007-01-02|   CBRENNAN|   2007-01-

In [37]:
instructor = spark.read\
    .option("header", "true")\
    .option("nullValue", "")\
    .option("inferSchema", "true")\
    .csv("student/INSTRUCTOR_DATA_TABLE.csv")\
    .withColumn("CREATED_DATE", to_date(col('CREATED_DATE'), "dd-MMM-yy"))\
    .withColumn("MODIFIED_DATE", to_date(col('MODIFIED_DATE'), "dd-MMM-yy"))

instructor.printSchema()
instructor.createOrReplaceTempView("instructor")
instructor.show(3)

root
 |-- INSTRUCTOR_ID: integer (nullable = true)
 |-- SALUTATION: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- STREET_ADDRESS: string (nullable = true)
 |-- ZIP: integer (nullable = true)
 |-- PHONE: integer (nullable = true)
 |-- CREATED_BY: string (nullable = true)
 |-- CREATED_DATE: date (nullable = true)
 |-- MODIFIED_BY: string (nullable = true)
 |-- MODIFIED_DATE: date (nullable = true)

+-------------+----------+----------+---------+--------------+-----+----------+----------+------------+-----------+-------------+
|INSTRUCTOR_ID|SALUTATION|FIRST_NAME|LAST_NAME|STREET_ADDRESS|  ZIP|     PHONE|CREATED_BY|CREATED_DATE|MODIFIED_BY|MODIFIED_DATE|
+-------------+----------+----------+---------+--------------+-----+----------+----------+------------+-----------+-------------+
|          101|        Mr|   Fernand|    Hanks| 100 East 87th|10015|2125551212|  ESILVEST|  2007-01-02|   ESILVEST|   2007-01-02|
|          10

In [38]:
student = spark.read\
    .option("header", "true")\
    .option("nullValue", "")\
    .option("inferSchema", "true")\
    .csv("student/STUDENT_DATA_TABLE.csv")\
    .withColumn("REGISTRATION_DATE", to_date(col('REGISTRATION_DATE'), "dd-MMM-yy"))\
    .withColumn("CREATED_DATE",      to_date(col('CREATED_DATE'), "dd-MMM-yy"))\
    .withColumn("MODIFIED_DATE",     to_date(col('MODIFIED_DATE'), "dd-MMM-yy"))

student.cache()
student.printSchema()
student.createOrReplaceTempView("student")
student.show(3)

root
 |-- STUDENT_ID: integer (nullable = true)
 |-- SALUTATION: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- STREET_ADDRESS: string (nullable = true)
 |-- ZIP: integer (nullable = true)
 |-- PHONE: string (nullable = true)
 |-- EMPLOYER: string (nullable = true)
 |-- REGISTRATION_DATE: date (nullable = true)
 |-- CREATED_BY: string (nullable = true)
 |-- CREATED_DATE: date (nullable = true)
 |-- MODIFIED_BY: string (nullable = true)
 |-- MODIFIED_DATE: date (nullable = true)

+----------+----------+----------+---------+------------------+-----+------------+---------------+-----------------+-----------+------------+-----------+-------------+
|STUDENT_ID|SALUTATION|FIRST_NAME|LAST_NAME|    STREET_ADDRESS|  ZIP|       PHONE|       EMPLOYER|REGISTRATION_DATE| CREATED_BY|CREATED_DATE|MODIFIED_BY|MODIFIED_DATE|
+----------+----------+----------+---------+------------------+-----+------------+---------------+----------------

In [39]:
enrollment = spark.read\
    .option("header", "true")\
    .option("nullValue", "")\
    .option("inferSchema", "true")\
    .csv("student/ENROLLMENT_DATA_TABLE.csv")\
    .withColumn("CREATED_DATE", to_date(col('CREATED_DATE'), "dd-MMM-yy"))\
    .withColumn("MODIFIED_DATE", to_date(col('MODIFIED_DATE'), "dd-MMM-yy"))

enrollment.cache()
enrollment.printSchema()
enrollment.createOrReplaceTempView("enrollment")
enrollment.show(3)

root
 |-- STUDENT_ID: integer (nullable = true)
 |-- SECTION_ID: integer (nullable = true)
 |-- ENROLL_DATE: string (nullable = true)
 |-- FINAL_GRADE: integer (nullable = true)
 |-- CREATED_BY: string (nullable = true)
 |-- CREATED_DATE: date (nullable = true)
 |-- MODIFIED_BY: string (nullable = true)
 |-- MODIFIED_DATE: date (nullable = true)

+----------+----------+-----------+-----------+----------+------------+-----------+-------------+
|STUDENT_ID|SECTION_ID|ENROLL_DATE|FINAL_GRADE|CREATED_BY|CREATED_DATE|MODIFIED_BY|MODIFIED_DATE|
+----------+----------+-----------+-----------+----------+------------+-----------+-------------+
|       215|       146|  13-FEB-07|       null|  DSCHERER|  2007-12-14|   BROSENZW|   2007-01-05|
|       215|       156|  13-FEB-07|       null|  DSCHERER|  2007-12-14|   BROSENZW|   2007-01-05|
|       216|       154|  13-FEB-07|       null|  DSCHERER|  2007-12-14|   BROSENZW|   2007-01-05|
+----------+----------+-----------+-----------+----------+-----

In [40]:
grade = spark.read\
    .option("header", "true")\
    .option("nullValue", "")\
    .option("inferSchema", "true")\
    .csv("student/GRADE_DATA_TABLE.csv")\
    .withColumn("CREATED_DATE", to_date(col('CREATED_DATE'), "dd-MMM-yy"))\
    .withColumn("MODIFIED_DATE", to_date(col('MODIFIED_DATE'), "dd-MMM-yy"))

grade.printSchema()
grade.createOrReplaceTempView("grade")
grade.show(3)

root
 |-- STUDENT_ID: integer (nullable = true)
 |-- SECTION_ID: integer (nullable = true)
 |-- GRADE_TYPE_CODE: string (nullable = true)
 |-- GRADE_CODE_OCCURRENCE: integer (nullable = true)
 |-- NUMERIC_GRADE: integer (nullable = true)
 |-- COMMENTS: string (nullable = true)
 |-- CREATED_BY: string (nullable = true)
 |-- CREATED_DATE: date (nullable = true)
 |-- MODIFIED_BY: string (nullable = true)
 |-- MODIFIED_DATE: date (nullable = true)

+----------+----------+---------------+---------------------+-------------+--------+----------+------------+-----------+-------------+
|STUDENT_ID|SECTION_ID|GRADE_TYPE_CODE|GRADE_CODE_OCCURRENCE|NUMERIC_GRADE|COMMENTS|CREATED_BY|CREATED_DATE|MODIFIED_BY|MODIFIED_DATE|
+----------+----------+---------------+---------------------+-------------+--------+----------+------------+-----------+-------------+
|       111|       133|             PA|                    6|           80|    null|  CBRENNAN|  2007-02-11|     JAYCAF|   2007-02-11|
|       111

Need to make sure 31-DEC-98 is converted to 1998-12-31, not 2098-12-31.

* [spark to_date function - how to convert 31-DEC-98 to 1998-12-31 not 2098-12-31](https://stackoverflow.com/questions/71182230)

In [41]:
grade_type = spark.read\
    .option("header", "true")\
    .option("nullValue", "")\
    .option("inferSchema", "true")\
    .csv("student/GRADE_TYPE_DATA_TABLE.csv")\
    .withColumn("CREATED_DATE", to_date(col('CREATED_DATE'), "dd-MMM-yy"))\
    .withColumn("MODIFIED_DATE", to_date(col('MODIFIED_DATE'), "dd-MMM-yy"))

grade_type.printSchema()
grade_type.createOrReplaceTempView("grade_type")
grade_type.show(3)

root
 |-- GRADE_TYPE_CODE: string (nullable = true)
 |-- DESCRIPTION: string (nullable = true)
 |-- CREATED_BY: string (nullable = true)
 |-- CREATED_DATE: date (nullable = true)
 |-- MODIFIED_BY: string (nullable = true)
 |-- MODIFIED_DATE: date (nullable = true)

+---------------+-----------+----------+------------+-----------+-------------+
|GRADE_TYPE_CODE|DESCRIPTION|CREATED_BY|CREATED_DATE|MODIFIED_BY|MODIFIED_DATE|
+---------------+-----------+----------+------------+-----------+-------------+
|             FI|      Final|  MCAFFREY|  1998-12-31|   MCAFFREY|   1998-12-31|
|             HM|   Homework|  MCAFFREY|  1998-12-31|   MCAFFREY|   1998-12-31|
|             MT|    Midterm|  MCAFFREY|  1998-12-31|   MCAFFREY|   1998-12-31|
+---------------+-----------+----------+------------+-----------+-------------+
only showing top 3 rows



In [42]:
zipcode = spark.read\
    .option("header", "true")\
    .option("nullValue", "")\
    .option("inferSchema", "true")\
    .csv("student/ZIPCODE_DATA_TABLE.csv")\
    .withColumn("CREATED_DATE", to_date(col('CREATED_DATE'), "dd-MMM-yy"))\
    .withColumn("MODIFIED_DATE", to_date(col('MODIFIED_DATE'), "dd-MMM-yy"))

zipcode.printSchema()
zipcode.createOrReplaceTempView("zipcode")
zipcode.show(3)

root
 |-- ZIP: integer (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- CREATED_BY: string (nullable = true)
 |-- CREATED_DATE: date (nullable = true)
 |-- MODIFIED_BY: string (nullable = true)
 |-- MODIFIED_DATE: date (nullable = true)

+-----+----------------+-----+----------+------------+-----------+-------------+
|  ZIP|            CITY|STATE|CREATED_BY|CREATED_DATE|MODIFIED_BY|MODIFIED_DATE|
+-----+----------------+-----+----------+------------+-----------+-------------+
|11101|Long Island City|   NY|  AMORRISO|  2007-08-03|   AMORRISO|   2007-11-24|
|11102|         Astoria|   NY|  AMORRISO|  2007-08-03|   AMORRISO|   2007-11-24|
|11103|         Astoria|   NY|  AMORRISO|  2007-08-03|   AMORRISO|   2007-11-24|
+-----+----------------+-----+----------+------------+-----------+-------------+
only showing top 3 rows



In [43]:
employee = spark.read\
    .option("header", "true")\
    .option("nullValue", "")\
    .option("inferSchema", "true")\
    .csv("student/EMPLOYEE_DATA_TABLE.csv")

employee.printSchema()
employee.createOrReplaceTempView("employee")
employee.show(3)

root
 |-- EMPLOYEE_ID: integer (nullable = true)
 |-- NAME: string (nullable = true)
 |-- SALARY: integer (nullable = true)
 |-- TITLE: string (nullable = true)

+-----------+------+------+---------+
|EMPLOYEE_ID|  NAME|SALARY|    TITLE|
+-----------+------+------+---------+
|          1|  John|  1000|  Analyst|
|          2|  Mary|  2000|  Manager|
|          3|Stella|  5000|President|
+-----------+------+------+---------+
only showing top 3 rows



---
# Pivot

Transform from Long/Stack Format to Wide/Unstack Format. 

1. Find the ID (keys) that identifyes an class instance that will be a row in Wide Format.
2. Create (ID, attribute, value) Long Format.
3. ```PIVOT (FOR attribute IN (<attributes>))```


* [PIVOT Clause](https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-pivot.html)

## Syntax
```
PIVOT (
    aggregate_expression(value) [ AS aggregate_expression_alias ]
    [ , ... ]   /* Continue with , for multipel aggregations */
    FOR attributes IN ( 
        attribute_list 
    )
) 
```


Find the number of classes starting at each location.

1. Find the ```location``` as **ID**.
2. Create (ID, attribute, value) Long Format table.
3. PIVOT (Long to Wide Transformation) that generates rows where each row is an class instance with attributes.


<img src="./image/pivot.jpg" align="left" width=600/>

In [73]:
query = """
SELECT
    location,
    date_format(start_date_time, "EEE") AS day_text,
    dayofweek(start_date_time) AS day_num
FROM
    section
ORDER BY location
"""
spark.sql(query).show()

+--------+--------+-------+
|location|day_text|day_num|
+--------+--------+-------+
|    H310|     Tue|      3|
|    L206|     Tue|      3|
|    L210|     Tue|      3|
|    L210|     Mon|      2|
|    L210|     Tue|      3|
|    L210|     Fri|      6|
|    L210|     Sat|      7|
|    L210|     Tue|      3|
|    L210|     Sat|      7|
|    L210|     Wed|      4|
|    L210|     Mon|      2|
|    L210|     Sun|      1|
|    L211|     Thu|      5|
|    L211|     Tue|      3|
|    L211|     Sat|      7|
|    L214|     Sat|      7|
|    L214|     Fri|      6|
|    L214|     Thu|      5|
|    L214|     Sun|      1|
|    L214|     Thu|      5|
+--------+--------+-------+
only showing top 20 rows



In [66]:
query = """
WITH location_day_cnt AS (
    SELECT
        location,
        upper(date_format(start_date_time, "EEE")) AS day,
        COUNT(*) AS cnt
    FROM
        section
    GROUP BY
        location, upper(date_format(start_date_time, "EEE"))
    ORDER BY location
)
SELECT
    *
FROM
    location_day_cnt
PIVOT (
    SUM(cnt)
    FOR day IN (
        'MON' AS MON,
        'TUE' AS TUE,
        'WED' AS WED,
        'THU' AS THU,
        'FRI' AS FRI,
        'SAT' AS SAT,
        'SUN' AS SUN
    )
)
ORDER BY
    location
"""
spark.sql(query).show()

+--------+----+----+----+----+----+----+----+
|location| MON| TUE| WED| THU| FRI| SAT| SUN|
+--------+----+----+----+----+----+----+----+
|    H310|null|   1|null|null|null|null|null|
|    L206|null|   1|null|null|null|null|null|
|    L210|   2|   3|   1|null|   1|   2|   1|
|    L211|null|   1|null|   1|null|   1|null|
|    L214|   2|   2|null|   2|   1|   4|   4|
|    L500|   1|   1|null|null|null|null|null|
|    L507|   4|   3|   3|null|   1|   3|   1|
|    L509|   4|   5|   3|   2|   1|   4|   6|
|    L511|null|null|null|null|null|   1|null|
|    M200|   1|null|null|null|null|null|null|
|    M311|   1|null|null|null|null|   1|   1|
|    M500|null|null|null|null|null|   1|null|
+--------+----+----+----+----+----+----+----+



---
# Stop Spark Session

In [ ]:
spark.stop()



# Cleanup

In [ ]:
del spark
gc.collect()